In [1]:
import gym
import click
import multiagent
import time
import tensorflow as tf
import make_env
import numpy as np
from rl.common.misc_util import set_global_seeds
from sandbox.mack.acktr_disc import Model, onehot
from sandbox.mack.acktr_disc_om import Model as Model_om
from sandbox.mack.policies import CategoricalPolicy
from sandbox.mack.policies_om import CategoricalPolicy as CategoricalPolicy_om
from sandbox.mack.opponent_policies import CategoricalPolicy as oppo_CategoricalPolicy
from rl import bench
import imageio
import seaborn as sns
import pickle as pkl
from sklearn.neighbors.kde import KernelDensity
import argparse

C:\Users\Renos\anaconda3\envs\TFCW\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.kde module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#@click.command()
#@click.option('--image', is_flag=True, flag_value=True)
#@click.option('--vis_dis', is_flag=True, flag_value=True)
#@click.option('--hyper_study', is_flag=True, flag_value=True)
#@click.option('--d', type=click.INT, default=1)
#@click.option('--g', type=click.INT, default=1)
#@click.option('--ent_coef', type=click.FLOAT, default=0)
#@click.option('--kl', is_flag=True, flag_value=True)
#@click.option('--env', type=click.STRING, default='simple_speaker_listener')
#@click.option('--algo', type=click.STRING, default='mack')
#@click.option('--disc_type', type=click.STRING, default='decentralized')
# @click.option('--path', type=click.STRING, default='./results/mack/simple_speaker_listener/l-0.1-b-1000/seed-1/checkpoint07000')
#@click.option('--seed', type=click.INT, default=1)
#@click.option('--epoch', type=click.INT, default=55000)
#@click.option('--sample', is_flag=True, flag_value=True)
#@click.option('--all_exp', is_flag=True, flag_value=True)
#@click.option('--curve', is_flag=True, flag_value=True)
#@click.option('--num_trajs', type=click.INT, default=100)
#@click.option('--traj_limitation', type=click.INT, default=200)


arg_image = True
arg_env = 'simple_tag'
arg_algo = 'mack'
arg_seed = 1
arg_epoch = 55000
arg_path = "tbd"
arg_num_trajs = 100

In [3]:
env_name = arg_env
env_id = arg_env

def create_env(seed):
    env = make_env.make_env(env_id)
    env.seed(seed)
    # env = bench.Monitor(env, '/tmp/',  allow_early_resets=True)
    set_global_seeds(seed)
    return env

env = create_env(0)
n_agents = len(env.action_space)

ob_space = env.observation_space
ac_space = env.action_space

n_actions = [action.n for action in ac_space]

WARN: Could not seed environment <MultiAgentEnv instance>



In [4]:
path = './results/'+arg_algo+'/'+env_name+'/'
if arg_algo in ['mack', 'mppo', 'mack_om']:
    path += '/l-0.1-b-1000/seed-'+str(1)+'/checkpoint' + '%05d'%arg_epoch
    
print(path)



./results/mack/simple_tag//l-0.1-b-1000/seed-1/checkpoint55000


In [5]:
make_model = lambda: Model(
    CategoricalPolicy, ob_space, ac_space, 1, total_timesteps=1e7, nprocs=2, nsteps=500,
    nstack=1, ent_coef=0.01, vf_coef=0.5, vf_fisher_coef=1.0, lr=0.01, max_grad_norm=0.5, kfac_clip=0.001,
    lrschedule='linear', identical=make_env.get_identical(env_id), use_kfac=False)
if arg_algo in ['mack_om', 'codail']:
    print("here for some reason")
    make_model = lambda: Model_om(
        CategoricalPolicy_om, oppo_CategoricalPolicy, ob_space, ac_space, 1, total_timesteps=1e7, nprocs=2, nsteps=500,
        nstack=1, ent_coef=0.01, vf_coef=0.5, vf_fisher_coef=1.0, lr=0.01, max_grad_norm=0.5, kfac_clip=0.001,
        lrschedule='linear', identical=make_env.get_identical(env_id), use_kfac=False)

model = make_model()
model.load(path)

images = []
sample_trajs = []
# num_trajs = 100
max_steps = 50
avg_ret = [[] for _ in range(n_agents)]

episode_r_n_all = []
episode_r_n_sum = []

all_pos = [[] for k in range(n_agents)]



[1, 2, 3, 4]



Tensor("policy_0/pi/add:0", shape=(1, 5), dtype=float32) Tensor("Placeholder_16:0", shape=(1,), dtype=int32)


Tensor("policy_0_1/pi/add:0", shape=(500, 5), dtype=float32) Tensor("Placeholder_20:0", shape=(500,), dtype=int32)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Tensor("policy_1/pi/add:0", shape=(1, 5), dtype=float32) Tensor("Placeholder_24:0", shape=(1,), dtype=int32)
Tensor("policy_1_1/pi/add:0", shape=(500, 5), dtype=float32) Tensor("Placeholder_28:0", shape=(500,), dtype=int32)
Tensor("policy_2/pi/add:0", shape=(1, 5), dtype=float32) Tensor("Placeholder_32:0", shape=(1,), dtype=int32)
Tensor("policy_2_1/pi/add:0", shape=(500, 5), dtype=float32) Tensor("Placeholder_36:0", shape=(500,), dtype=int32)
Tensor("policy_3/pi/add:0", shape=(1, 5), dtype=float32) Tensor("Placeholder_40:0", shape=(1,), dtype=int32)
Tensor("policy_3_1/pi/add:0", shape=(500, 5), dtype=

In [6]:
obs = env.reset()

for i in range(arg_num_trajs):
    print("trajectory ", i)
    all_ob, all_agent_ob, all_ac, all_rew, ep_ret = [], [], [], [], [0 for k in range(n_agents)]
    for k in range(n_agents):
        all_ob.append([])
        all_ac.append([])
        all_rew.append([])
    obs = env.reset()
    obs = [ob[None, :] for ob in obs]
    action = [np.zeros([1]) for _ in range(n_agents)]
    step = 0
    done = False
    episode_r_n = [0.] * n_agents
    while (not done) or (step<=2):
        if arg_algo != "random":
            action, _, _ = model.step(obs, action)
            actions_list = [onehot(action[k][0], n_actions[k]) for k in range(n_agents)]
        else:
            actions_list = [onehot(np.random.randint(n_actions[k]), n_actions[k]) for k in range(n_agents)]

        for k in range(n_agents):
            all_ob[k].append(obs[k])
            all_ac[k].append(actions_list[k])
        all_agent_ob.append(np.concatenate(obs, axis=1))
        obs, rew, done, _ = env.step(actions_list)
        print(obs)
        episode_r_n = [episode_r_n[i]+rew[i] for i in range(n_agents)]
        for k in range(n_agents):
            all_rew[k].append(rew[k])
            ep_ret[k] += rew[k]
            all_pos[k].append(np.array(env.world.agents[k].state.p_pos))

        obs = [ob[None, :] for ob in obs]
        step += 1

        if arg_image:
            img = env.render(mode='rgb_array')
            images.append(img[0])
            time.sleep(0.02)
        if step == max_steps or True in done:
            done = True
            # step = 0
        else:
            done = False

    # print("\n--- episode-{} | [sum-reward]: {}".format(i, np.sum(episode_r_n)))
    episode_r_n_all.append(episode_r_n)
    episode_r_n_sum.append(np.sum(episode_r_n))

    for k in range(n_agents):
        all_ob[k] = np.squeeze(all_ob[k])

    all_agent_ob = np.squeeze(all_agent_ob)
    traj_data = {
        "ob": all_ob, "ac": all_ac, "rew": all_rew,
        "ep_ret": ep_ret, "all_ob": all_agent_ob
    }

    sample_trajs.append(traj_data)
    # print('episode', i, 'expected_return', ep_ret)

    for k in range(n_agents):
        avg_ret[k].append(ep_ret[k])

In [7]:
data_path = './results/'+arg_algo+'/'+env_name+'/l-0.1-b-1000/seed-'+str(1)+ '/checkpoint' + '%05d'%arg_epoch + "-%d"%arg_num_trajs + "tra-1.pkl"

In [8]:
with open(data_path, 'rb') as f:
    data = pkl.load(f)

In [9]:
data[0]["ob"][3][0][2:4]

array([-0.52792787,  0.79273633])

In [10]:
env.world.entities

In [11]:
def update_env_uppos_tag(state_data, timestep):
    pray_perspective = state_data[3][timestep]
    
    pray_pos = pray_perspective[2:4]
    landmark_1_pos = pray_perspective[4:6] + pray_pos
    landmark_2_pos = pray_perspective[6:8] + pray_pos
    predator_1_pos = pray_perspective[8:10] + pray_pos
    predator_2_pos = pray_perspective[10:12] + pray_pos
    predator_3_pos = pray_perspective[12:14] + pray_pos
    
    return [predator_1_pos, predator_2_pos, predator_3_pos, pray_pos, landmark_1_pos,landmark_2_pos]
    
    

In [13]:
obs = env.reset()
images = []
for i in range(50):
    if(arg_env == "simple_tag"):
        update_list = update_env_uppos_tag(data[0]["ob"],i)
    for j, pos in enumerate(update_list):
        env.world.entities[j].state.p_pos = pos
        
        
    img = env.render(mode='rgb_array')
    images.append(img[0])
    time.sleep(0.05)

